# Lezhin 작품별 썸네일

In [ ]:
import pandas as pd
from selenium import webdriver
import urllib.request
import urllib
import time
import re
import os
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [ ]:
URL = 'https://www.lezhin.com/ko/scheduled?day=1'

driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=1)
driver.get(URL)
time.sleep(1)

#요일별 데이터 가져오기
for i in range(1,9): 

    driver.find_element_by_xpath('//*[@id="main"]/div/a['+str(i)+']').click()
    time.sleep(0.5)

    for key_down in range(0,5):
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    webtoon_area = soup.select('#main > ul.lzComic__list')

    #제목 수집
    title = soup.find_all('div', {'class': 'lzComic__title'}) # 제목 수집
    titles = []
    for t in range(len(title)):
        titles.append(re.sub('[^0-9a-zA-Zㄱ-힗]','',title[t].get_text()))

    titles = titles[0:]
    print(len(titles))

    #이미지 수집
    data_list=soup.findAll('div',{'class':'lzComic__thumb'})

    li_list = []
    for data in data_list:
        li_list.extend(data.findAll('picture')) #해당 부분을 찾아 li_list와 병합

    thumb = []

    for li in li_list:
        img = li.find('img')
        img_src = img['src']
        thumb.append(img_src)
    thumb = thumb[0:]
    print(len(thumb))

    for k in range(len(thumb)):
            try:
                urllib.request.urlretrieve(thumb[k], './LZ/imgs/{}.webp'.format(titles[k]))
            except:
                k += 1
                continue
            k += 1

    driver.execute_script("window.scrollTo(0, -10000)")
                                   
driver.close()
print('Complete')

In [ ]:
print(titles)

In [ ]:
lz = pd.DataFrame({'title':titles, 'thumb':thumb})
lz = lz.drop_duplicates(['title'], keep='first')

In [ ]:
lz

In [ ]:
lz.to_csv('LZ_list.csv', encoding='euc-kr')

## PIL 활용하여 이미지 변환 저장

In [ ]:
from PIL import Image

im = Image.open('LZ/imgs/1미터사이.webp').convert('RGB')
im.save('1미터사이.png', 'png')

In [ ]:
data = './LZ/imgs/'
c_data = './LZ/imgs_c/'
file_list = os.listdir(data)
webp_list = []

k = 0
for i in file_list:
    k += 1
    if k%10==0:
        print('변환 개수 :' +str(k)+' / '+str(len(file_list)))
    img = Image.open(data+str(i))
    img_1 = img.convert('RGB')
    webp_list.append(img_1)
img_1.save(c_data, 'png', save_all=True, append_images=webp_list)
print('Complete')

## Lezhin 회차별 썸네일

In [ ]:
URL = 'https://www.lezhin.com/ko/comic/'
link = input('작품 링크 : ')

driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=1)
driver.get(URL+link)
time.sleep(1)

body = driver.find_element_by_tag_name('body')
last = driver.execute_script('return document.body.scrollHeight')

# while True:
#     scroll_down = 0
#     while scroll_down < 10:
#         elem.send_keys(Keys.PAGE_DOWN)
#         time.sleep(0.5)
#         scroll_down += 1

for key_down in range(1):
    # driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

    for scroll in range(0, 40):     # PAGE_DOWN 횟수
        scroll_down = 0
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.15)
        scroll_down += 1

        new = driver.execute_script('return document.body.scrollHeight')

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    webtoon_area = soup.select('#main > #section > ul.comic-episode-list')

    #제목 수집
    title = soup.find_all('div', {'class': 'episode__name'}) # 제목 수집
    titles = []
    for t in range(len(title)):
        titles.append(re.sub('[^0-9a-zA-Zㄱ-힗]','',title[t].get_text()))

    titles = titles[0:]
    print(len(titles))

    #이미지 수집
    data_list=soup.findAll('div',{'class':'episode__thumb'})

    li_list = []
    for data in data_list:
        li_list.extend(data.findAll('picture')) #해당 부분을 찾아 li_list와 병합

    thumb = []

    for li in li_list:
        img = li.find('img')
        img_src = img['src']
        thumb.append(img_src)
    thumb = thumb[0:]
    print(len(thumb))

    for k in range(len(thumb)):
            try:
                urllib.request.urlretrieve(thumb[k], './LZ/imgs_t/{}.png'.format(titles[k]))       # 저장형식 지정
            except:
                k += 1
                continue
            k += 1

    if new == last:
        print('PAGE COMPLETE')      # 스크롤 범위 지정

    last = new

driver.close()
print('Complete')

28
28
PAGE COMPLETE
Complete


In [ ]:
thumb

['https://ccdn.lezhin.com/v2/comics/4566161041522688/episodes/4807617213759488/images/cover.jpg?updated=1587792744678&width=156',
 'https://ccdn.lezhin.com/v2/comics/4566161041522688/episodes/5626063272935424/images/cover.jpg?updated=1587792760645&width=156',
 'https://ccdn.lezhin.com/v2/comics/4566161041522688/episodes/5228262602964992/images/cover.jpg?updated=1587792772804&width=156',
 'https://ccdn.lezhin.com/v2/comics/4566161041522688/episodes/6559526832439296/images/cover.jpg?updated=1587792785132&width=156',
 'https://ccdn.lezhin.com/v2/comics/4566161041522688/episodes/4729939483820032/images/cover.jpg?updated=1587792798194&width=156',
 'https://ccdn.lezhin.com/v2/comics/4566161041522688/episodes/6363082880057344/images/cover.jpg?updated=1587793188583&width=156',
 'https://ccdn.lezhin.com/v2/comics/4566161041522688/episodes/5282174299144192/images/cover.jpg?updated=1587793205244&width=156',
 'https://ccdn.lezhin.com/v2/comics/4566161041522688/episodes/5355961686294528/images/cove